In [3]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(tsibble, warn.conflicts = FALSE)
library(zeallot, warn.conflicts = FALSE)
library(sf, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(assertr, warn.conflicts = FALSE)
options(repr.plot.width = 9, repr.plot.res = 300)
library(vroom, warn.conflicts = FALSE)

source("notebooks/integrazioni_regionali/nb_tools/state_avail.R")
source("notebooks/integrazioni_regionali/nb_tools/pairing_procedure.R")
source("src/load/load.R")
source("src/analysis/data/clim_availability.R")
source("src/analysis/data/quality_check.R")
source("src/pairing/analysis.R")
source("src/pairing/matching.R")
source("src/pairing/plots.R")
source("src/pairing/combining.R")
source("src/pairing/displaying.R")
source("src/plot/plot_helpers.R")
source("src/database/write.R")
source("src/database/open.R")

first_date <- as.Date("2000-01-01")
last_date <- as.Date("2022-12-31")


In [2]:
path.veneto <- file.path(path.ds, "ARPA", "VENETO", "Dati_ARPAV_STAZIONI_AUTOMATICHE_al_02_02_2022")
path.tmin <- file.path(path.veneto, "ARPAV_Tmin.csv")
path.tmax <- file.path(path.veneto, "ARPAV_Tmax.csv")


In [34]:
tab <- vroom(path.tmin, n_max = 8L, col_names = FALSE) |> select(!X2:X4)
col_names <- tab$X1
ntab <- tab |> select(-X1) |> t()
colnames(ntab) <- col_names
ntab <- as_tibble(ntab) |> rename(original_id = statcd, station_name = statnm, elevation = altitude, province = provsgl, start_date = iniziovalidita, end_date = finevalidita)

Rows: 8 Columns: 259
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (256): X1, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X...
lgl   (3): X2, X3, X4

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [38]:
ntab |> mutate(start_date = as.Date(start_date), end_date = na_if(end_date, "attiva") |> as.Date())

original_id,station_name,xkmpos,ykmpos,elevation,province,start_date,end_date,start_date_psd,end_date_psd
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
3,Arabba,1720711.174,5153644.921,1642.0,BL,1984-01-01 00:00:00,attiva,1984-01-01,NA
4,Belluno Viale Europa,1746856.0,5113883.0,392.0,BL,1991-05-01 00:00:00,2006-06-07 00:00:00,1991-05-01,2006-06-07
9,Caprile,1729706.442,5147353.788,1007.0,BL,1984-03-01 00:00:00,attiva,1984-03-01,NA
10,Selva di Cadore,1733606.0,5148934.0,1494.0,BL,1984-07-04 00:00:00,2001-11-28 00:00:00,1984-07-04,2001-11-28
11,Malga Ciapela,1723206.0,5145802.0,1475.0,BL,1985-01-01 00:00:00,attiva,1985-01-01,NA
13,Biois a Cencenighe,1728263.0,5137495.0,770.0,BL,1984-02-15 00:00:00,2010-09-29 00:00:00,1984-02-15,2010-09-29
15,Gares,1721980.0,5132902.0,1365.0,BL,1984-03-16 00:00:00,attiva,1984-03-16,NA
16,Sappade Falcade,1721636.0,5139182.0,1321.0,BL,1984-02-15 00:00:00,2002-10-01 00:00:00,1984-02-15,2002-10-01
17,Perarolo,1758079.0,5143342.0,525.0,BL,2002-12-18 00:00:00,attiva,2002-12-18,NA
